In [86]:
import torch
from torch.utils.data import Dataset
import os
import meshio
import numpy as np

class VesselDatasetSinglePoint(Dataset):
    def __init__(self, data_path, num_fluid_samples=1024, num_meshpoints=1024, seed=None, save_tensors=False, load_tensors=False):
        if seed is not None:
            torch.manual_seed(seed)
            np.random.seed(seed)
        else:
            torch.seed()
            np.random.seed()
            
        self.data_path = data_path
        self.vessel_files = self._get_vessel_files()
        self.num_vessels = len(self.vessel_files)
        self.num_fluid_samples = num_fluid_samples
        
        if save_tensors:
            self.mesh_points_tensor, self.fluid_points_tensor, self.sys_vel_tensor = self._get_vessel_tensor(num_meshpoints, num_fluid_samples)
            torch.save(self.mesh_points_tensor, '/home/ne34gux/workspace/vascunet/data/torch_tensors/mesh_points.pt')
            torch.save(self.fluid_points_tensor, '/home/ne34gux/workspace/vascunet/data/torch_tensors/fluid_points.pt')
            torch.save(self.sys_vel_tensor, '/home/ne34gux/workspace/vascunet/data/torch_tensors/sys_vel.pt')
            
        if load_tensors:
            self.mesh_points_tensor = torch.load('/home/ne34gux/workspace/vascunet/data/torch_tensors/mesh_points.pt')
            self.fluid_points_tensor = torch.load('/home/ne34gux/workspace/vascunet/data/torch_tensors/fluid_points.pt')
            self.sys_vel_tensor = torch.load('/home/ne34gux/workspace/vascunet/data/torch_tensors/sys_vel.pt')
        else:
            self.mesh_points_tensor, self.fluid_points_tensor, self.sys_vel_tensor = self._get_vessel_tensor(num_meshpoints, num_fluid_samples)

    def __len__(self):
        return self.num_vessels * self.num_fluid_samples

    def __getitem__(self, idx):
        vessel_idx = idx % self.num_vessels
        sample_idx = idx // self.num_vessels

        return_dict = {
            'mesh_tensor': self.mesh_points_tensor[vessel_idx],
            'fluid_points': self.fluid_points_tensor[vessel_idx,sample_idx],
            'sys_vel_tensor': self.sys_vel_tensor[vessel_idx,sample_idx]
        }    
        
        return return_dict
    
    def _get_vessel_tensor(self, num_mesh_points, num_fluid_points):
        mesh_points_list = []
        fluid_points_list = []
        sys_vel_list = []
        for vf in self.vessel_files:
            fluid_points = torch.Tensor(meshio.read(vf['fluid']).points)
            mesh_points = torch.Tensor(meshio.read(vf['mesh']).points)
            sys_vel = torch.Tensor(meshio.read(vf['fluid']).point_data['velocity_systolic'])
            
            idx1 = np.random.choice(np.arange(len(mesh_points)), num_mesh_points, replace=False)
            idx2 = np.random.choice(np.arange(len(fluid_points)), num_fluid_points, replace=False)
            
            mesh_points_list.append(mesh_points[idx1])
            fluid_points_list.append(fluid_points[idx2])
            sys_vel_list.append(sys_vel[idx2])

        mesh_points_tensor = torch.stack(mesh_points_list)
        fluid_points_tensor = torch.stack(fluid_points_list)
        sys_vel_tensor = torch.stack(sys_vel_list)
        
        return mesh_points_tensor, fluid_points_tensor, sys_vel_tensor
    
    def _get_vessel_files(self):
        vessel_files = []
        for fluid_filename in os.listdir(self.data_path):
            if fluid_filename.endswith('fluid.vtu'):
                fluid_file_path = os.path.join(self.data_path, fluid_filename)
                mesh_file_path = fluid_file_path.replace('fluid.vtu', 'wss.vtu')
                vessel_files.append({
                    'fluid': fluid_file_path,
                    'mesh': mesh_file_path
                })
        vessel_files = np.array(vessel_files)
            
        return vessel_files

In [87]:
import utils.helper_functions as hf
DATA_DIR = hf.get_project_root() / "data" / "carotid_flow_database"

In [88]:
dataset = VesselDatasetSinglePoint(data_path=DATA_DIR, num_fluid_samples=1024, num_meshpoints=1024, seed=42, save_tensors=False, load_tensors=True)

In [89]:
len(dataset), dataset[0]

(134144,
 {'mesh_tensor': tensor([[-12.0038, -16.7095, -43.4129],
          [-18.5062, -14.8418, -33.0287],
          [-15.7870,  -9.2905, -30.7143],
          ...,
          [-16.4697,  -9.2046, -30.7808],
          [ -8.8081, -23.6444, -58.6591],
          [-23.7843,  -7.8213, -25.8157]]),
  'fluid_points': tensor([-18.3106, -15.8157, -39.8416]),
  'sys_vel_tensor': tensor([0.0644, 0.2314, 0.4049])})

In [90]:
dataset[0]['mesh_tensor'].shape, dataset[0]['fluid_points'].shape, dataset[0]['sys_vel_tensor'].shape

(torch.Size([1024, 3]), torch.Size([3]), torch.Size([3]))

In [91]:
dataset[0]['mesh_tensor'], dataset[0]['fluid_points'], dataset[0]['sys_vel_tensor']

(tensor([[-12.0038, -16.7095, -43.4129],
         [-18.5062, -14.8418, -33.0287],
         [-15.7870,  -9.2905, -30.7143],
         ...,
         [-16.4697,  -9.2046, -30.7808],
         [ -8.8081, -23.6444, -58.6591],
         [-23.7843,  -7.8213, -25.8157]]),
 tensor([-18.3106, -15.8157, -39.8416]),
 tensor([0.0644, 0.2314, 0.4049]))

In [92]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [93]:
for batch in dataloader:
    print(batch)
    break

{'mesh_tensor': tensor([[[  15.0654,   30.5976,  -95.7510],
         [  20.4908,   28.9421,  -97.0862],
         [  18.6837,   17.3482,  -80.4057],
         ...,
         [  21.4984,   22.2529,  -79.0030],
         [  15.9567,   17.7851, -100.1871],
         [  18.2287,   25.2290,  -72.4033]],

        [[ -36.9287,  -32.5856, -326.6594],
         [ -35.8058,  -33.7285, -324.4643],
         [ -27.1111,  -18.3628, -351.1126],
         ...,
         [ -34.8822,  -32.7306, -323.3774],
         [ -36.0574,  -39.0164, -313.2820],
         [ -37.5345,  -31.7609, -325.1357]],

        [[  18.9705,   49.8970,  -77.0496],
         [  17.0981,   46.3736,  -52.8206],
         [  19.7162,   46.5903,  -79.4324],
         ...,
         [  18.0991,   38.9845, -101.5038],
         [  18.2949,   50.7728,  -45.9139],
         [  23.3627,   44.7012,  -57.6003]],

        ...,

        [[  43.7294,   22.4511,  -38.6308],
         [  40.5658,   30.3632,  -32.3161],
         [  51.8068,   12.4532,  -58.3801]

In [94]:
mesh_points = batch['mesh_tensor']
fluid_points = batch['fluid_points']
sys_vel = batch['sys_vel_tensor']

input_tensor = torch.concatenate((fluid_points.reshape(-1, 1, 3), mesh_points), dim=1)

#torch.concatenate((mesh_points, fluid_points), dim=1).shape

mesh_points.shape, fluid_points.shape, sys_vel.shape, input_tensor.shape

(torch.Size([8, 1024, 3]),
 torch.Size([8, 3]),
 torch.Size([8, 3]),
 torch.Size([8, 1025, 3]))

In [27]:
fluid_points.reshape(-1, 1, 3)

tensor([[[0.6164, 0.9935, 0.5608]],

        [[0.2975, 0.2308, 0.9917]],

        [[0.3502, 0.0175, 0.9995]],

        [[0.8352, 0.8584, 0.7134]],

        [[0.4849, 0.9666, 0.3584]],

        [[0.9914, 0.7098, 0.4048]],

        [[0.9702, 0.9644, 0.0694]],

        [[0.0222, 0.3187, 0.9966]]])

In [31]:
import lightning as L
import torch
from torch import optim, nn

class VesselModelSinlgePoint1(L.LightningModule):
    def __init__(self, learning_rate=1e-3, in_features=1025, out_features=3, batch_size=2):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features*3, 512),
            nn.Tanh(),
            nn.Linear(512, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 16),
            nn.Tanh(),
            nn.Linear(16, 8),
            nn.Tanh(),
            nn.Linear(8, out_features),
        )
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.out_features = out_features

    def training_step(self, batch, batch_idx): 
        mesh_points = batch['mesh_tensor']
        fluid_points = batch['fluid_points']
        sys_vel = batch['sys_vel_tensor']

        input_tensor = torch.concatenate((fluid_points.reshape(-1, 1, 3), mesh_points), dim=1) 
        pred = self.model(input_tensor)
        #loss = nn.functional.mse_loss(pred, sys_vel)
        loss = torch.sum(torch.abs(pred - sys_vel))
        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True, batch_size=self.batch_size)
        return loss    
        
    def validation_step(self, batch, batch_idx):
        mesh_points = batch['mesh_tensor']
        fluid_points = batch['fluid_points']
        sys_vel = batch['sys_vel_tensor']

        input_tensor = torch.concatenate((fluid_points.reshape(-1, 1, 3), mesh_points), dim=1) 
        pred = self.model(input_tensor)
        #loss = nn.functional.mse_loss(pred, sys_vel)
        loss = torch.sum(torch.abs(pred - sys_vel))
        self.log("val_loss", loss, prog_bar=True, on_step=True, on_epoch=True, batch_size=self.batch_size)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def forward(self, x):
        x = torch.stack([x])
        x = self.model(x)
        x = x.reshape((self.out_features, 3))
        return x.detach()

In [35]:
mesh_points = batch['mesh_tensor']
fluid_points = batch['fluid_points']
sys_vel = batch['sys_vel_tensor']

input_tensor = torch.concatenate((fluid_points.reshape(-1, 1, 3), mesh_points), dim=1)

model = VesselModelSinlgePoint1()

pred = model(input_tensor)

loss = nn.functional.mse_loss(pred, sys_vel)
#torch.concatenate((mesh_points, fluid_points), dim=1).shape

mesh_points.shape, fluid_points.shape, sys_vel.shape, input_tensor.shape, pred.shape, loss

(torch.Size([8, 1024, 3]),
 torch.Size([8, 3]),
 torch.Size([8, 3]),
 torch.Size([8, 1025, 3]),
 torch.Size([8, 3]),
 tensor(0.1374, grad_fn=<MseLossBackward0>))

In [2]:
from vessel_dataloader import VesselDatasetSinglePoint
import utils.helper_functions as hf

DATA_DIR = hf.get_project_root() / "data" / "carotid_flow_database"

dataset = VesselDatasetSinglePoint(
    DATA_DIR, 
    num_fluid_samples=8_192,
    num_meshpoints=4_096,
    seed=666,
    save_tensors=False,
    load_tensors=True,
)

In [3]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset, 
    batch_size=2, 
    shuffle=True, 
    num_workers=2, 
)

In [4]:
for batch in dataloader:
    print(batch)
    break

{'mesh_tensor': tensor([[[0.5566, 0.6797, 0.8257],
         [0.1236, 0.7021, 0.8896],
         [0.1136, 0.8576, 0.6683],
         ...,
         [0.2516, 0.7353, 0.3908],
         [0.4983, 0.6573, 0.7836],
         [0.5245, 0.5850, 0.1677]],

        [[0.8359, 0.7577, 0.8889],
         [0.5278, 0.2143, 0.0224],
         [0.7475, 0.7764, 0.8904],
         ...,
         [0.5397, 0.4119, 0.7594],
         [0.2550, 0.3503, 0.3841],
         [0.8179, 0.6874, 0.7047]]]), 'fluid_points': tensor([[ 13.3566,  50.0063, -64.0705],
        [-40.3676,  -5.4390,  54.2405]]), 'sys_vel_tensor': tensor([[-0.0085,  0.0166, -0.0339],
        [ 0.1438,  0.1013,  0.0959]])}


In [5]:
print(batch['mesh_tensor'].shape)
print(batch['fluid_points'].shape)
print(batch['sys_vel_tensor'].shape)

torch.Size([2, 4096, 3])
torch.Size([2, 3])
torch.Size([2, 3])


In [2]:
from vessel_dataloader import VesselAutoencoderDataset
import utils.helper_functions as hf


DATA_DIR = hf.get_project_root() / "data" / "carotid_flow_database"

dataset = VesselAutoencoderDataset(
    DATA_DIR, 
    num_points=200_000,
    apply_point_restrictions=False
)

from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset, 
    batch_size=1, 
    shuffle=False, 
    num_workers=2, 
)

for batch in dataloader:
    #print(batch)
    break

print(batch[0].shape)
print(batch[1].shape)

#print(batch['fluid_points'].shape)
#print(batch['sys_vel_tensor'].shape)

torch.Size([1, 121439, 3])
torch.Size([1, 121439, 3])


In [3]:
import meshio
import torch

vessel_file= dataset._get_vessel_files()[0]

mesh_data = meshio.read(vessel_file['mesh'])
fluid_data = meshio.read(vessel_file['fluid'])
len_data = fluid_data.points.shape[0] + mesh_data.points.shape[0]
#idx = np.random.choice(np.arange(len_data), self.num_points, replace=False)
points = torch.concat([torch.Tensor(fluid_data.points), torch.Tensor(mesh_data.points)], axis=0)

points_min = points.min(dim=0, keepdim=True)[0]
points_max = points.max(dim=0, keepdim=True)[0]
points = (points - points_min) / (points_max - points_min)

velocities = torch.concat([torch.Tensor(fluid_data.point_data['velocity_systolic']), torch.zeros(mesh_data.points.shape)], axis=0)

points.shape, velocities.shape, points[:5], velocities[:5]

(torch.Size([121439, 3]),
 torch.Size([121439, 3]),
 tensor([[0.4637, 0.8142, 0.9164],
         [0.4641, 0.8135, 0.9161],
         [0.4630, 0.8132, 0.9162],
         [0.4630, 0.8145, 0.9161],
         [0.4636, 0.8141, 0.9155]]),
 tensor([[-0.1221,  0.0485,  0.2049],
         [-0.1064,  0.0374,  0.1774],
         [-0.0281,  0.0081,  0.0430],
         [-0.0568,  0.0264,  0.0940],
         [-0.0396,  0.0132,  0.0704]]))

In [4]:
import numpy as np
from scipy.interpolate import griddata

def interpolate_vectors_to_grid(points, velocities, grid_shape, method='linear'):
    """
    Interpolates velocity vectors at 3D points into a 3D grid of a predefined shape.

    Parameters:
    - points: A numpy array of shape (N, 3), where N is the number of points, representing the 3D coordinates.
    - velocities: A numpy array of shape (N, 3), where N is the number of points, representing the velocity vectors at these points.
    - grid_shape: A tuple of 3 integers defining the shape of the 3D grid (depth, height, width).
    - method: Interpolation method. Options include 'linear', 'nearest', and 'cubic'.

    Returns:
    - A numpy array of shape (grid_shape[0], grid_shape[1], grid_shape[2], 3) representing the interpolated velocity vectors on the 3D grid.
    """
    
    # Generate linearly spaced points for each axis based on the bounds and the desired shape
    x = np.linspace(np.min(points[:,0]), np.max(points[:,0]), grid_shape[0])
    y = np.linspace(np.min(points[:,1]), np.max(points[:,1]), grid_shape[1])
    z = np.linspace(np.min(points[:,2]), np.max(points[:,2]), grid_shape[2])

    # Create a 3D grid from the 1D arrays
    grid_x, grid_y, grid_z = np.meshgrid(x, y, z, indexing='ij')
    
    # Interpolate the velocity vectors onto the grid
    grid_vx = griddata(points, velocities[:,0], (grid_x, grid_y, grid_z), method=method, fill_value=0)
    grid_vy = griddata(points, velocities[:,1], (grid_x, grid_y, grid_z), method=method, fill_value=0)
    grid_vz = griddata(points, velocities[:,2], (grid_x, grid_y, grid_z), method=method, fill_value=0)
    
    # Combine the interpolated velocities into a single array
    grid_velocities = np.stack((grid_vx, grid_vy, grid_vz), axis=-1)
    
    return grid_velocities

In [6]:
import plotly.graph_objects as go

keep_every = 50

fig = go.Figure(data = go.Cone(
    x=points[::keep_every, 0],
    y=points[::keep_every, 1],
    z=points[::keep_every, 2],
    u=velocities[::keep_every, 0],
    v=velocities[::keep_every, 1],
    w=velocities[::keep_every, 2],
    colorscale='Viridis',
    sizemode="scaled",
    sizeref=5
))


fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='Velocities of the origial Vessel',
    width=800,
    height=800
)

fig.show()

In [7]:
interpolated_velocities = interpolate_vectors_to_grid(np.array(points), np.array((velocities)), (64, 64, 64), method='linear')

In [8]:
import plotly.graph_objects as go
import numpy as np

# Assuming interpolated_velocities is your 3D grid of velocity vectors
# Let's create a grid of points for demonstration. You should replace these with your actual grid points.
x = np.linspace(0, 1, interpolated_velocities.shape[0])
y = np.linspace(0, 1, interpolated_velocities.shape[1])
z = np.linspace(0, 1, interpolated_velocities.shape[2])
x, y, z = np.meshgrid(x, y, z)

# Flatten the arrays for plotting
x, y, z = x.flatten(), y.flatten(), z.flatten()
u, v, w = interpolated_velocities[:,:,:,0].flatten(), interpolated_velocities[:,:,:,1].flatten(), interpolated_velocities[:,:,:,2].flatten()

# Filter out zero vectors to reduce clutter, adjust the condition as needed
mask = (u**2 + v**2 + w**2) > 0.1
x, y, z, u, v, w = x[mask], y[mask], z[mask], u[mask], v[mask], w[mask]

fig = go.Figure(data=go.Cone(x=x, y=y, z=z, u=v, v=u, w=w, colorscale='Viridis', sizemode='scaled', sizeref=2))

fig.update_layout(width=1000, height=1000,
                  scene=dict(aspectmode='cube'),
                  title="3D Vector Field Visualization")

fig.show()

In [ ]:
vessel_mask = (interpolated_velocities[:,:,:,0]**2 + interpolated_velocities[:,:,:,1]**2 + interpolated_velocities[:,:,:,2]**2) > 0.1

In [9]:
import numpy as np

import plotly.graph_objects as go

# Assuming vessel_mask is your 3D vessel mask
# Let's create a grid of points for demonstration. You should replace these with your actual grid points.
x = np.linspace(0, 1, vessel_mask.shape[0])
y = np.linspace(0, 1, vessel_mask.shape[1])
z = np.linspace(0, 1, vessel_mask.shape[2])
x, y, z = np.meshgrid(x, y, z)

# Flatten the arrays for plotting
x, y, z = x.flatten(), y.flatten(), z.flatten()
mask = vessel_mask.flatten()

# Filter out non-vessel points
x, y, z = x[mask], y[mask], z[mask]

fig = go.Figure(data=go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2)))

fig.update_layout(width=800, height=800,
                  scene=dict(aspectmode='cube'),
                  title="Vessel Mask in 3D")

fig.show()

: 

In [6]:
def get_vessel_grid_data(batch, size=(64, 64, 64), method='linear', threashold=0.1):
    points, velocities = batch
    interpolated_velocities = interpolate_vectors_to_grid(
        np.array(points), 
        np.array((velocities)), 
        size, 
        method=method
    )
    vessel_mask = np.sum(interpolated_velocities**2, axis=-1) > threashold
    return vessel_mask, interpolated_velocities

In [10]:
get_vessel_grid_data(batch)